In [58]:
import sklearn
import numpy as np
from random import randrange
from keras import layers, models, optimizers, backend, metrics, callbacks
import codecs
from keras.preprocessing import text, sequence
from nltk.tokenize import word_tokenize
import random

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 12) # set default size of plots

In [59]:
import csv

Решим задачу классификации текстовых данных. В качестве датасета возьмём базу рецензий сайта IMDB, рецензии размечены на два класса: позитивные и негативные. Такая задача называется sentiment analysis

1. Считаем данные из CSV файла

In [3]:
texts = []
labels = []
with open('labeledTrainData.tsv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    _ = next(reader)
    for l in reader:
        texts.append(l[2])
        labels.append(l[1])

Убедимся, что данные считались корректно:

In [4]:
texts[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

Теперь необходимо перевести текст в такое представление, с которым нам будет удобно работать. Существует модель [мешка слов](https://en.wikipedia.org/wiki/Bag-of-words_model), которая долгое время использовалась в классических методах. К сожалению, эта модель не учитывает семантическую информацию и векторы, присваиваемые словам, имеют большую размерность, что делает её не лучшим выбором для тренировки нейронной сети.

Мы будем пользоваться [word embeddings](https://en.wikipedia.org/wiki/Word_embedding), специальными векторами рассчитанными таким образом, чтобы учитывать сематническую информацию и при этом иметь небольшой размер. Подробно про рассчёт embeddings на примере Word2Vec можно прочесть в [википедии](https://en.wikipedia.org/wiki/Word2vec), [оригинальносй статье](https://arxiv.org/abs/1301.3781) или слайдах курса.

Существует несколько видов embedding'ов. Мы будем пользоваться [Glove](https://nlp.stanford.edu/pubs/glove.pdf). Скачайте векторы размерности 100 c сайта https://nlp.stanford.edu/projects/glove/ и положите в директорию с блокнотом. 


![w2v.png](w2v.png)

Vocab это вспомогательный класс, помогающий работать с вокабуляром. Внутри находятся два словаря, в одном хранится соответсвие между словами и индексами в glove, а в другом -- между индексами и словами.

In [5]:
class Vocab(object):
    w2i = None
    i2w = None

Следующая функция загружает Glove с диска. Обратите внимание, что добавляются два специальных ветора: один (PAD) отвечает за отступ, а другой (UNK) -- за неизвестное слово.

In [6]:
def load_glove(filename, embedding_size=300):
    vocabulary_mapping = {'PAD': 0}
    temp = np.zeros(embedding_size)
    embeddings_list = [[0] * embedding_size]
    id = 1
    with codecs.open(filename, 'rb', 'utf-8') as glove_file:
        for line in glove_file:
            items = line.strip().split()

            if len(items) != embedding_size + 1:
                continue

            token = items[0]
            embedding = np.array([float(i) for i in items[1:]])
            temp += embedding
            embeddings_list.append(embedding)
            vocabulary_mapping[token] = id
            id += 1

    vocabulary_mapping['<UNK>'] = id
    temp /= len(embeddings_list) - 2
    embeddings_list.append(temp)
    embeddings = np.array(embeddings_list)
    vocab = Vocab()
    vocab.i2w = {v: k for k, v in vocabulary_mapping.items()}
    vocab.w2i = vocabulary_mapping
    print('glove was loaded')
    return embeddings, vocab

Загрузим векторы с диска

In [7]:
embeddings, vocab = load_glove('glove.6B.100d.txt', 100)

glove was loaded


Теперь напишем функцию, которая будет разбивать предложения на токены (слова), а затем каждому токену ставить в соответсвие индекс вектора из Glove. Ограничим марсимальный размер текста 128 словами (из соображений скорости вычислений)

In [8]:
def sent_to_id_vec(sent, vocab, max_len=128, mode='tokenize'):
    sent = sent.lower()

    if mode == 'tokenize':
        tokens = word_tokenize(sent)
    elif mode == 'split':
        tokens = sent.split()
    else:
        raise Error(f'Unknown mode: {mode}')

    if max_len is not None and len(tokens) > max_len:
        tokens = (t for i, t in enumerate(tokens) if i < max_len)

    result = []

    for token in tokens:
        if token in vocab.w2i:
            result.append(vocab.w2i[token])
        else:
            result.append(vocab.w2i['<UNK>'])

    return result

Векторизуем наш датасет:

In [9]:
sequences = []
for t in texts:
    temp = sent_to_id_vec(t, vocab)
    sequences.append(temp)

Теперь в каждом объекте датасета находится не текст, а последовательность идентификаторов.

In [10]:
sequences[0]

[18,
 65,
 38,
 3497,
 223,
 136,
 23,
 1,
 1601,
 18,
 56359,
 42,
 463,
 552,
 6147,
 5,
 27,
 404,
 2,
 2642,
 1,
 6555,
 3831,
 188,
 6,
 64,
 2,
 3137,
 1,
 46975,
 6,
 3137,
 211141,
 379,
 3,
 1882,
 42,
 121,
 304,
 5,
 170,
 8,
 1193,
 8918,
 76,
 38,
 1857,
 39,
 42,
 805,
 16,
 589,
 3452,
 7,
 1,
 34440,
 121,
 5,
 1882,
 160,
 61,
 193,
 1677,
 402,
 19,
 15,
 2174,
 47,
 4115,
 3,
 211141,
 15,
 154,
 3211,
 2,
 154,
 2048,
 320,
 43,
 42,
 2716,
 223,
 5,
 254,
 23,
 1,
 5993,
 62,
 21,
 16,
 1448,
 357,
 3,
 78,
 4,
 21,
 32,
 9328,
 3977,
 60,
 56359,
 10,
 2519,
 1812,
 1,
 429,
 6,
 53,
 1,
 4399,
 1444,
 4,
 1604,
 33,
 979,
 400001,
 19796,
 30411,
 275,
 12258,
 19796,
 30411,
 275,
 12258,
 16675,
 4572,
 35,
 4]

Проверим, что мы не ошиблись и выполним обратное преобразование для произвольного предложения.

In [11]:
' '.join([vocab.i2w[i] for i in sequences[0]])

"with all this stuff going down at the moment with mj i 've started listening to his music , watching the odd documentary here and there , watched the wiz and watched moonwalker again . maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent . moonwalker is part biography , part feature film which i remember going to see at the cinema when it was originally released . some of it has subtle messages about mj 's feeling towards the press and also the obvious message of drugs are bad <UNK> < br / > < br / > visually impressive but of"

Предложения совпадают, значит мы сделали верные преобразования

Для решения проблемы, воспользуемся рекеррентной нейронной сетью

![rnn.png](rnn.png)

Рекуррентные сети способны улавливать зависимости между словами и чувствительны к порядку. 

Однако, наивная реализация RNN-ячейки не способна показать сколько-нибудь значимые результаты. Воспользуемся ячейкой специального вида, называющейся LSTM. Про LSTM можно прочесть [в блоге Криса Ола](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) и слайдах лекций.



In [12]:
embeddings.shape

(400002, 100)

In [13]:
def build_LSTM_classifier():
    # Точка входа в граф задаётся при помощи специальных тензоров типа Input
    # Первая координата соответсвует длине текста, так как тексты в датасете имеют разную длину
    # значение считается переменным
    text_input = layers.Input(shape=(None,), dtype='int32')
    
    # Создаём специальный слой для работы с embedding, 
    # Его функция -- заменять индентификатор вектором из Glove
    # Указываем trainable = False, чтобы векторы embedding'ов не изменялись в процессе обучения
    embedding_layer = layers.Embedding(input_dim = embeddings.shape[0], 
                                       output_dim = embeddings.shape[1], 
                                       weights=[embeddings],
                                       mask_zero=True,
                                       trainable = False)
    
    x = embedding_layer(text_input)
    
    # Создаём рекуррентную ячейку
    # Первый параметр отвечает за размер внутреннего состояния (памяти ячейки)
    # По умолчанию такой слой возвращает только последнее состояние (см. картинку),
    # Если мы хотим получить состояния на каждом шаге необходимо указать return_sequences = True
    x = layers.LSTM(256, recurrent_dropout=0.25)(x)
    
    # Полученный результат направляем в полносвязный слой, который будет осуществлять классификацию
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=text_input, outputs=output, name = 'LSTM_classifier')
    
    
    #  Для оптимизации будем использовать Adam 
    adam = optimizers.Adam(lr=0.0001)
    
    #Перед испльзованием модель необходимо скомпилировать
    model.compile(adam, 'binary_crossentropy', metrics=['acc'])
    
    return model 

In [14]:
backend.clear_session()
model = build_LSTM_classifier()

Выведем информацию по модели

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 100)         40000200  
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 40,366,025
Trainable params: 365,825
Non-trainable params: 40,000,200
_________________________________________________________________


Разобьём датасет на три части

In [16]:
def split_train_val(train_size = 0.6, val_size = 0.1, test_size = 0.3):
    boundary_train = int(len(sequences) * train_size)
    boundary_val = int(len(sequences) * (train_size + val_size))
    
    train_set = (sequences[:boundary_train], labels[:boundary_train])
    val_set = (sequences[boundary_train:boundary_val], labels[boundary_train:boundary_val])
    test_set = (sequences[boundary_val:], labels[boundary_val:])
    
    return train_set, val_set, test_set

In [17]:
(x_train, y_train), (x_val, y_val), (x_test, y_tes) = split_train_val()

Почти всё готово, чтобы начать обучение. Но так, как все предлдожения разной длины мы не можем конвертировать x в тензор, нам необходимо выровнять длину. Для этого мы воспользуемся специальной функцией pad_sequences(), доступной в keras. Недостающие элементы будут заполнены специальным символом PAD

Но нам не хотелось бы увеличивать все элементы в датасете по размеру максимального. По этогому будем генерировать батчи сами и применять pad_sequences к ним независимо

In [18]:
def generate_batches(x, y, batch_size=64):
    i = 0
    while True:
        i = i % len(x)
        yield sequence.pad_sequences(x[i:i+batch_size]), y[i:i+batch_size]
        i += batch_size

In [19]:
train_generator = generate_batches(x_train, y_train)
val_generator = generate_batches(x_train, y_train)

Теперь обучим нашу модель

In [20]:
cbs = [
    callbacks.ModelCheckpoint('models/LSTM/{epoch:02d}-{val_loss:.4f}.h5', save_best_only=True),
    callbacks.TensorBoard(log_dir='models/LSTM/'),
    callbacks.ReduceLROnPlateau(factor=0.1, patience=3, verbose=1, min_lr=0.00001, epsilon=0.01)]


model.fit_generator(generator= train_generator, 
                    validation_data = val_generator, 
                    validation_steps= 40,
                    steps_per_epoch=256,
                    epochs=20,
                    callbacks=cbs)

Epoch 1/20
256/256 [==============================] - 90s 353ms/step - loss: 0.6518 - acc: 0.6062 - val_loss: 0.5865 - val_acc: 0.6914
Epoch 2/20
256/256 [==============================] - 90s 350ms/step - loss: 0.5668 - acc: 0.7133 - val_loss: 0.5554 - val_acc: 0.7215
Epoch 3/20
256/256 [==============================] - 89s 349ms/step - loss: 0.5476 - acc: 0.7247 - val_loss: 0.5222 - val_acc: 0.7445
Epoch 4/20
256/256 [==============================] - 89s 349ms/step - loss: 0.5326 - acc: 0.7347 - val_loss: 0.5458 - val_acc: 0.7336
Epoch 5/20
256/256 [==============================] - 89s 347ms/step - loss: 0.5209 - acc: 0.7438 - val_loss: 0.6306 - val_acc: 0.7095
Epoch 6/20
256/256 [==============================] - 90s 350ms/step - loss: 0.5207 - acc: 0.7440 - val_loss: 0.5247 - val_acc: 0.7551
Epoch 7/20
256/256 [==============================] - 89s 349ms/step - loss: 0.5061 - acc: 0.7558 - val_loss: 0.4683 - val_acc: 0.7820
Epoch 8/20
256/256 [==============================] - 8

In [22]:
model.evaluate(sequence.pad_sequences(x_test), y=y_tes, batch_size=64)

7500/7500 [==============================] - 14s 2ms/step


[0.46759841016133624, 0.77959999998410545]

77% Неплохой результат, но можно лучше.

Проблема в том, что RNN забывают начало последовательности, а в нашем датасете все рецензии достаточно велики. Есть способ уменьшить влияние этого эффекта -- обучить два LSTM, один идёт от начала к концу предложения, а другой -- от конца к началу. Такой подход называется bidirectional

In [43]:
def build_biLSTM_classifier():
    text_input = layers.Input(shape=(None,), dtype='int32')
    

    embedding_layer = layers.Embedding(input_dim = embeddings.shape[0], 
                                       output_dim = embeddings.shape[1], 
                                       weights=[embeddings],
                                       mask_zero=True,
                                       trainable = False)
    
    x = embedding_layer(text_input)

    # Всё что нужно изменить. В keras есть специальный слой для реализации bidirectional моделей
    x = layers.Bidirectional(layers.LSTM(256, recurrent_dropout=0.25))(x)
    
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=text_input, outputs=output, name = 'LSTM_classifier')
    
    
    adam = optimizers.Adam(lr=0.0001)
    
    model.compile(adam, 'binary_crossentropy', metrics=['acc'])
    
    return model 

In [44]:
backend.clear_session()
model = build_biLSTM_classifier()

In [45]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 100)         40000200  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               731136    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 40,731,849
Trainable params: 731,649
Non-trainable params: 40,000,200
_________________________________________________________________


In [48]:
cbs = [
    callbacks.ModelCheckpoint('models/biLSTM/{epoch:02d}-{val_loss:.4f}.h5', save_best_only=True),
    callbacks.TensorBoard(log_dir='models/biLSTM/'),
    callbacks.ReduceLROnPlateau(factor=0.1, patience=3, verbose=1, min_lr=0.00001, epsilon=0.01)]


model.fit_generator(generator= train_generator, 
                    validation_data = val_generator, 
                    validation_steps= 40,
                    steps_per_epoch=128,
                    epochs=20,
                    callbacks=cbs)

Epoch 1/20
128/128 [==============================] - 65s 507ms/step - loss: 0.4256 - acc: 0.8004 - val_loss: 0.4060 - val_acc: 0.8121
Epoch 2/20
128/128 [==============================] - 63s 488ms/step - loss: 0.4223 - acc: 0.8052 - val_loss: 0.4241 - val_acc: 0.8063
Epoch 3/20
128/128 [==============================] - 66s 518ms/step - loss: 0.4094 - acc: 0.8157 - val_loss: 0.4375 - val_acc: 0.7918
Epoch 4/20
128/128 [==============================] - 66s 515ms/step - loss: 0.4126 - acc: 0.8124 - val_loss: 0.3751 - val_acc: 0.8313
Epoch 5/20
128/128 [==============================] - 66s 512ms/step - loss: 0.3997 - acc: 0.8181 - val_loss: 0.4413 - val_acc: 0.7805
Epoch 6/20
128/128 [==============================] - 66s 512ms/step - loss: 0.4080 - acc: 0.8123 - val_loss: 0.3937 - val_acc: 0.8207
Epoch 7/20
128/128 [==============================] - 68s 527ms/step - loss: 0.3892 - acc: 0.8251 - val_loss: 0.3745 - val_acc: 0.8405
Epoch 8/20
127/128 [============================>.] - E

In [57]:
model.evaluate(sequence.pad_sequences(x_test), y=y_tes, batch_size=64)

7500/7500 [==============================] - 18s 2ms/step


[0.45073856875101725, 0.79440000003178912]

Стало лучше, но вероятно дополнительные проценты можно получить продолжив тренировку, а так же убрав ограничение на максимальную длину последовательности или увеличив ёмкость сети. **Поэкспериментируйте!**

Важно помнить, что RNN не являются единственной моделью для работы с последовательностями (и текстом в частности). Например, существую эфективные [модели, основанные на свёртках](https://arxiv.org/abs/1612.08083).